In [1]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [2]:
import numpy as np
import glob
import pandas as pd
import cv2

In [3]:
def Model_one(input_shape, out_dims = 5):

    model = Sequential()
    model.add(Convolution2D(32, kernel_size=(3, 3), padding='same', input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(strides=(2, 2)))
    
    model.add(Convolution2D(64, kernel_size=(3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(strides=(2, 2)))
    
    model.add(Convolution2D(128, kernel_size=(3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(strides=(2, 2)))
    
    model.add(Convolution2D(256, kernel_size=(3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(strides=(2, 2)))
    
    model.add(Convolution2D(256, kernel_size=(3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(strides=(2, 2), name='transfer_barrier'))
    
    model.add(Flatten()) 
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(out_dims))
    model.add(Activation('softmax'))

    return model

In [4]:
def tn_image_batch(folder, N):
    
    images = glob.glob(folder + '/*')
    N = min(N, len(images))
    random_images = np.random.choice(np.arange(len(images)), N, replace=False)
    data = pd.DataFrame(columns=['20', '50', '100', '200', '500', '1000', '2000', '5000'])
    X_data = []
    for i in random_images:
        image = images[i]
        img = load_img(image)
        X_data.append(img_to_array(img))
        data = data.append({image.split('/')[-1].split('_')[0]: 1}, ignore_index=True)
    data = data.fillna(0)
    X_data = np.array(X_data)
    X_data = X_data / 255
    data = np.array(data)
    
    return X_data, data

In [5]:
model = Model_one((224, 224, 3), 8)
#model.load_weights('tt_model.h5')

In [21]:
model.load_weights('tt_model.h5')
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
import cv2

In [26]:
x, y = tn_image_batch('data/test', 100)

In [28]:
pred = model.predict(x)

In [27]:
model.evaluate(x, y)

100/100 [==============================] - 1s 5ms/step


[1.1283122062683106, 0.86]

In [13]:
coins = ['20', '50', '100', '200', '500', '1000', '2000', '5000']

In [31]:
c = 1

print(coins[np.argmax(pred[c])])
print(coins[np.argmax(y[c])])
cv2.imshow('image', x[c])
cv2.waitKey(0)
cv2.destroyAllWindows()

5000
5000
